In [3]:
import pandas as pd

## read file
def read_file(year,month,day):
    data = []
    if len(str(month)) == 1:
        month_ = '0'+str(month)
    else:
        month_ = str(month)
    if len(str(day)) == 1:
        day_ = '0'+str(day)
    else:
        day_ = str(day)
    datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_'+str(year)+'.'+str(month_)+'.'+str(day_)+'.csv'
    data = pd.read_csv(datapath)
    data_CN14 = data[data.Series == data['Series'][0]]
    data = data_CN14
    return data

In [4]:
data = read_file(2014,1,30)

In [64]:
import time 
import csv
with open('order_book_2014_1_30.csv', 'wb') as csvfile:
    f = csv.writer(csvfile)
def insert(order_book_data,data_to_insert,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position:]
    return pd.concat((top,data_to_insert,bottom)).reset_index(drop = True)
def draw_out(order_book_data,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position+1:]
    return pd.concat((top,bottom)).reset_index(drop = True)
data['QuantityDifference_'] = data['QuantityDifference']
data_ask = data[(data.BidOrAsk == 'A')].reset_index(drop=True)
data_bid = data[(data.BidOrAsk == 'B')].reset_index(drop=True)
start  =  time.time()
order_book_bid = []
order_book_ask = []
timestamp = data.TimeStamp[0]
temp_ask = 0
temp_bid = 0
bid = []
ask = []
timestamp_ = []
x1 = data[(data.BidOrAsk == 'A')].TimeStamp.unique()
x2 = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
y = data[data['TimeStamp'].str.contains(timestamp)]
bid.append(y[(y.BidOrAsk == 'B')][["Price","OrderNumber","QuantityDifference","QuantityDifference_"]]) # bid
ask.append(y[(y.BidOrAsk == 'A')][["Price","OrderNumber","QuantityDifference","QuantityDifference_"]]) # ask
a = bid[0].sort(['Price'],ascending = [False])
b = ask[0].sort(['Price'],ascending = [True])    
order_book_bid.append(a[a.QuantityDifference != 0].reset_index(drop = True))
order_book_ask.append(b[b.QuantityDifference != 0].reset_index(drop = True))
order_book_bid.append(0)
order_book_ask.append(0)
order_book_bid_sum = order_book_bid[0][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False,sort = False).sum()
order_book_ask_sum = order_book_ask[0][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()
order_book_bid_ask = pd.concat([order_book_bid_sum[['Price','QuantityDifference']],order_book_ask_sum[['Price','QuantityDifference']]],axis = 1)

with open('order_book.csv','a') as f:
    order_book = csv.writer(f)
    order_book.writerow(["Bid","Bid_Quantity","Ask","Ask_Quantity"])
    order_book.writerow(["TimeStamp",timestamp])
    order_book = csv.writer(f,delimiter=',')
    for i in range(0,max(len(order_book_ask[0]),len(order_book_bid[0]))):
        order_book.writerow(order_book_bid_ask[i:i+1].values.tolist()[0])
        

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:30: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:31: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [65]:
def order_book_to_csv(order_book_bid,order_book_ask,data,i):
    order_book_bid_sum = order_book_bid[['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False,sort = False).sum()
    order_book_ask_sum = order_book_ask[['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()
    order_book_bid_ask = pd.concat([order_book_bid_sum[['Price','QuantityDifference']],order_book_ask_sum[['Price','QuantityDifference']]],axis = 1)    
    #print order_book_bid_ask
    with open('order_book_2014_1_30.csv','a') as f:
        order_book = csv.writer(f)
        order_book.writerow(["TimeStamp",data.TimeStamp[i-1:i].iloc[0]])
        order_book = csv.writer(f,delimiter=',')
        for i in range(0,len(order_book_bid_ask)):
            order_book.writerow(order_book_bid_ask[i:i+1].values.tolist()[0])
    return order_book_bid_sum,order_book_ask_sum

In [66]:
for i in range(len(y),5000,1):
    print i#,temp_bid,temp_ask
    time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                  int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                  int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
    if time_second == 32400:
        order_book_bid[temp_bid] = order_book_bid[temp_bid].sort(['Price'],ascending = [False]).reset_index(drop = True)
        order_book_ask[temp_ask] = order_book_ask[temp_ask].sort(['Price'],ascending = [True]).reset_index(drop = True)
        pass 
    if data.BidOrAsk[i] == 'A':
        data_ask_Quantity = data.BestQuantity[i]
        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            if order_book_bid[temp_bid].Price[0] >= data[i:i+1].Price.iloc[0] and time_second < 32400:
                for k in range(0,len(order_book_bid[temp_bid])):
                    diff = order_book_bid[temp_bid].QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                    if order_book_bid[temp_bid].Price[k] >= data[i:i+1].Price.iloc[0] and diff >= 0:
                        order_book_bid[temp_bid].QuantityDifference_[k] = diff
                        data[i:i+1].QuantityDifference_.iloc[0] = 0
                        break
                    elif order_book_bid[temp_bid].Price[k] >= data[i:i+1].Price.iloc[0] and diff < 0:
                        order_book_bid[temp_bid].QuantityDifference_[k] = 0
                        data[i:i+1].QuantityDifference_.iloc[0] = -diff
                        pass
                    else:
                        break
            if data.TimeStamp[i] == x1[temp_ask]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                length_order_book_zero = len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price <= data['Price'][i:i+1].iloc[0]) & (order_book_ask[temp_ask].QuantityDifference_ == 0)])
                if time_second > 32400:
                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:  
                            print 'Some error1(Ask & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< (len(order_book_ask[temp_ask])-1):
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:#or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error1(Ask & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'   
                            pass
                    elif position_ == len(order_book_ask[temp_ask])-1:
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error1(Ask & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True' 
                        pass
                else:
                    print 'before open market'
            elif data.TimeStamp[i] != x1[temp_ask]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                if temp_ask == 0:
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = order_book_ask[temp_ask - 1]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                  
                    if time_second > 32400:
                        if position_ == 0 and len(order_book_ask[0]) > 1:
                            if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 

                                print 'Some error2(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                break
                            else:
                                print 'position and Best Price is True'                     
                                pass
                        elif 0 < position_< len(order_book_ask[temp_ask])-1:
                            if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ -  1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 

                                print 'Some error2(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                                break
                            else:
                                print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_ask[temp_ask])-1:   
                            if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 

                                print 'Some error2(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            else:
                                print 'position and Best Price is True'
                                pass
                        elif position_ == 0 and len(order_book_ask[0]) == 1:
                            print 'position and Best Price is True'
                            pass
                    else:
                        print 'before open market'
                else:
                    order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)
                    if time_second > 33400:
                        if data[i-1:i].BidOrAsk.iloc[0] == 'A':
                            if order_book_ask_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]:
                                print 'Best ask quantity is ture'
                                pass
                            else:
                                print 'Best ask quantity is false'
                                break 
                        else:
                            j = i - 1
                            while j >= 1:
                                print j
                                print data[j-1:j].BidOrAsk.iloc[0]
                                if data[j-1:j].BidOrAsk.iloc[0] == 'A':
                                    if order_book_ask_sum[0:1].values.tolist()[0][1] == data[j-1:j].BestQuantity.iloc[0]:
                                        print 'Best ask quantity is ture'
                                        break
                                else:
                                    j = j - 1
                                    pass   
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    order_book_ask.append(order_book_ask[temp_ask])
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)#int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                    if time_second > 32400:
                        if position_ == 0:
                            if order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                                print 'Some error3(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                break
                            else:
                                print 'position and Best Price is True'                     
                                pass
                        elif 0 < position_< len(order_book_ask[temp_ask])-1:
                            if order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                                print 'Some error3(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                                break
                            else:
                                print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_ask[temp_ask])-1:    
                            if order_book_ask[temp_ask][position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 

                                print 'Some error3(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            else:
                                print 'position and Best Price is True'                        
                                pass
                        elif position_ == 0 and len(order_book_ask[0]) == 1:
                            print 'position and Best Price is True'
                            pass
                    else:
                        print 'before open market'
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    
            if data.TimeStamp[i] == x1[temp_ask]: 
                print 'timestamp = %s'%(data.TimeStamp[i])
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                if time_second > 32400:
                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                            print 'Some error4(Ask & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass       
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                            print 'Some error4(Ask & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_ask[temp_ask])-1:       
                        if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 

                            print 'Some error4(Ask & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])   
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            pass                        
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        pass
                else:
                    print 'before open market'                   
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
            elif data.TimeStamp[i] != x1[temp_ask]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)
                if time_second > 33400:
                    if data[i-1:i].BidOrAsk.iloc[0] == 'A':
                        if order_book_ask_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]:
                            print 'Best ask quantity is ture'
                            pass
                        else:
                            print 'Best ask quantity is false'
                            break 
                    else:
                        j = i - 1
                        while j >= 1:
                            print j
                            print data[j-1:j].BidOrAsk.iloc[0]
                            if data[j-1:j].BidOrAsk.iloc[0] == 'A':
                                if order_book_ask_sum[0:1].values.tolist()[0][1] == data[j-1:j].BestQuantity.iloc[0]:
                                    print 'Best ask quantity is ture'
                                    break
                            else:
                                j = j - 1
                                pass
                order_book_ask.append(order_book_ask[temp_ask])
                order_number_ =  data['OrderNumber'][i : i + 1].iloc[0]
                position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_ask = temp_ask + 1
                if time_second > 32400:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                            print 'Some error5(Ask & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                            print 'Some error5(Ask & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_ask[temp_ask])-1:

                        if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 

                            print 'Some error5(Ask & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            pass                       
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        pass      
                else:
                    print 'before open market'
                    
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                    
    elif data.BidOrAsk[i] == 'B':
        data_bid_Quantity = data.BestQuantity[i]
        if int(data[['QuantityDifference']][i:i+1].values) > 0: 
            if order_book_ask[temp_ask].Price[0] <= data[i:i+1].Price.iloc[0] and time_second < 32400:
                for k in range(0,len(order_book_ask[temp_ask])):
                    diff = order_book_ask[temp_ask].QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                    if order_book_ask[temp_ask].Price[k] <= data[i:i+1].Price.iloc[0] and diff >= 0:
                        order_book_ask[temp_ask].QuantityDifference_[k] = diff
                        data[i:i+1].QuantityDifference_.iloc[0] = 0
                        break
                    elif order_book_ask[temp_ask].Price[k] <= data[i:i+1].Price.iloc[0] and diff < 0:
                        order_book_ask[temp_ask].QuantityDifference_[k] = 0
                        data[i:i+1].QuantityDifference_.iloc[0] = - diff
                        pass
                    else:
                        break
            if data.TimeStamp[i] == x2[temp_bid]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                length_order_book_zero = len(order_book_bid[temp_bid][(order_book_bid[temp_bid].Price >= data['Price'][i:i+1].iloc[0]) & (order_book_bid[temp_bid].QuantityDifference_ == 0)])
                if time_second > 32400:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:                
                            print 'Some error1(Bid & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass     
                    elif 0 < position_< len(order_book_bid[temp_bid])-1: 
                        if order_book_bid[temp_bid][position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error1(Bid & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break    
                        else:
                            print 'position and Best Price is True'   
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1 and len(order_book_bid[0]) > 1:      
                        if order_book_bid[temp_bid][position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                            print 'Some error1(Bid & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                
                            pass
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        pass 
                else:
                    print 'before open market'
                    pass
            elif data.TimeStamp[i] != x2[temp_bid]:
                print temp_bid
                print 'timestamp = %s'%(data.TimeStamp[i])
                if temp_bid == 0:
                    best_price = data[i:(i+1)]['BestPrice']             
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = order_book_bid[temp_bid - 1]
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                 
                    if time_second > 32400:
                        if position_ == 0  and len(order_book_bid[0]) > 1:
                            if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                print 'Some error2(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                break
                            else:
                                print 'position and Best Price is True'                     
                                pass       
                        elif 0 < position_< len(order_book_bid[temp_bid])-1:
                            if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                                print 'Some error2(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_bid[temp_bid])-1:    
                            if order_book_bid[temp_bid][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                print 'Some error2(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])                                        
                                pass
                            else:
                                print 'position and Best Price is True'
                                pass
                        elif position_ == 0 and len(order_book_bid[0]) == 1:
                                print 'position and Best Price is True'
                                pass
                    else:
                        print 'before open market'
                        pass
                else:
                    if time_second > 33400:
                        order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)                    
                        if data[i-1:i].BidOrAsk.iloc[0] == 'B':
                            if order_book_bid_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]:
                                print 'Best bid quantity is ture'
                                pass
                            else:
                                print 'Best bid quantity is false'
                                break 
                        else:
                            j = i - 1
                            while j >= 1:
                                print j
                                print data[j-1:j].BidOrAsk.iloc[0]
                                if data[j-1:j].BidOrAsk.iloc[0] == 'B':
                                    if order_book_bid_sum[0:1].values.tolist()[0][1] == data[j-1:j].BestQuantity.iloc[0]:
                                        print 'Best bid quantity is ture'
                                        break
                                    else:
                                        print 'Best bid quantity is false'
                                        break
                                else:
                                    j = j - 1
                                    pass
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    order_book_bid.append(order_book_bid[temp_bid])
                    temp_bid = temp_bid + 1 
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)   
                    if time_second > 32400:
                        if position_ == 0 and len(order_book_bid[0]) > 1:
                            if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                print 'Some error3(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                break
                            else:
                                print 'position and Best Price is True'                     
                                pass
                        elif 0 < position_< len(order_book_bid[temp_bid])-1:
                            if order_book_bid[temp_bid][position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                                print 'Some error3(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_bid[temp_bid])-1:
                            if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                print 'Some error3(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            else:
                                print 'position and Best Price is True'                        
                                pass
                        elif position_ == 0 and len(order_book_bid[0]) == 1:
                            print 'position and Best Price is True'
                            pass
                    else:
                        print 'before open market'
                        pass
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    
            if data.TimeStamp[i] == x2[temp_bid]: 
                print 'timestamp = %s'%(data.TimeStamp[i])
                order_number_ =  data['OrderNumber'][i : i + 1].iloc[0]
                position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                if time_second > 32400:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error4(Bid & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass               
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error4(Bid & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                            print 'Some error4(Bid & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            pass
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        pass
                else:
                    print 'before open market'
                    pass
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]                
            elif data.TimeStamp[i] != x2[temp_bid]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                if time_second > 33400:
                    order_book_bid_sum,order_book_ask_sum = order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)
                    if data[i-1:i].BidOrAsk.iloc[0] == 'B':
                        if order_book_bid_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]:
                            print 'Best bid quantity is ture'
                            pass
                        else:
                            print 'Best bid quantity is false'
                            break 
                    else:
                        j = i - 1
                        while j >= 1:
                            print j
                            print data[j-1:j].BidOrAsk.iloc[0]
                            if data[j-1:j].BidOrAsk.iloc[0] == 'B':
                                if order_book_bid_sum[0:1].values.tolist()[0][1] == data[j-1:j].BestQuantity.iloc[0]:
                                    print 'Best bid quantity is ture'
                                    break
                                else:
                                    print 'Best bid quantity is false'
                                    break
                            else:
                                j = j - 1
                                pass 
                order_book_bid.append(order_book_bid[temp_bid])
                order_number_ =  data['OrderNumber'][i:i+1].iloc[0]
                position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_bid = temp_bid + 1
                if time_second > 32400:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error5(Bid & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error5(Bid & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1: 
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                            print 'Some error5(Bid & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            pass
                        else:
                            print 'position and Best Price is True'                        
                            pass    
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        pass  
                else:
                    print 'before open market'
                    pass
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
end = time.time()            
print "Total time = %f"%(end - start)

13
0
timestamp = 2014-01-30D08:45:00.361550600
before open market
14
1
timestamp = 2014-01-30D08:45:01.844526600
before open market
15
2
timestamp = 2014-01-30D08:45:01.858526600
before open market
16
3
timestamp = 2014-01-30D08:45:01.887526600
before open market
17
4
timestamp = 2014-01-30D08:45:01.924526600
before open market
18
5
timestamp = 2014-01-30D08:45:01.982526600
before open market
19
timestamp = 2014-01-30D08:45:02.493519800
before open market
20
timestamp = 2014-01-30D08:45:26.558230200
before open market
21
7
timestamp = 2014-01-30D08:45:30.856178600
before open market
22
8
timestamp = 2014-01-30D08:45:31.064178600
before open market
23
9
timestamp = 2014-01-30D08:45:44.716010600
before open market
24
timestamp = 2014-01-30D08:45:45.136010600
before open market
25
timestamp = 2014-01-30D08:45:45.136010600
before open market
26
timestamp = 2014-01-30D08:45:52.934914600
before open market
27
12
timestamp = 2014-01-30D08:45:53.613906000
before open market
28
13
timestamp = 2

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:269: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


22
timestamp = 2014-01-30D08:46:15.379650600
before open market
67
timestamp = 2014-01-30D08:46:16.764626600
before open market
68
timestamp = 2014-01-30D08:46:17.823614600
before open market
69
timestamp = 2014-01-30D08:46:22.630562100
before open market
70
23
timestamp = 2014-01-30D08:46:26.474518600
before open market
71
timestamp = 2014-01-30D08:46:26.640513900
before open market
72
24
timestamp = 2014-01-30D08:46:39.409362600
before open market
73
25
timestamp = 2014-01-30D08:46:44.307302600
before open market
74
26
timestamp = 2014-01-30D08:46:46.771266600
before open market
75
timestamp = 2014-01-30D08:46:50.157230600
before open market
76
timestamp = 2014-01-30D08:46:50.322230600
before open market
77
29
timestamp = 2014-01-30D08:46:55.298170600
before open market
78
30
timestamp = 2014-01-30D08:47:06.746031400
before open market
79
31
timestamp = 2014-01-30D08:47:14.583942600
before open market
80
32
timestamp = 2014-01-30D08:47:21.128858600
before open market
81
33
timestamp 

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


timestamp = 2014-01-30D08:47:35.170690600
before open market
84
35
timestamp = 2014-01-30D08:47:35.721690600
before open market
85
36
timestamp = 2014-01-30D08:47:39.914630600
before open market
86
timestamp = 2014-01-30D08:47:42.530606600
before open market
87
timestamp = 2014-01-30D08:47:44.426582600
before open market
88
37
timestamp = 2014-01-30D08:47:46.683558600
before open market
89
timestamp = 2014-01-30D08:48:06.078318600
before open market
90
38
timestamp = 2014-01-30D08:48:06.575318600
before open market
91
timestamp = 2014-01-30D08:48:06.659318600
before open market
92
39
timestamp = 2014-01-30D08:48:15.087210600
before open market
93
timestamp = 2014-01-30D08:48:18.056174600
before open market
94
40
timestamp = 2014-01-30D08:49:09.953523400
before open market
95
41
timestamp = 2014-01-30D08:49:19.444396600
before open market
96
42
timestamp = 2014-01-30D08:49:19.553396600
before open market
97
43
timestamp = 2014-01-30D08:49:25.315318600
before open market
98
timestamp = 2

...

timestamp = 2014-01-30D09:00:00.049089600
before open market
799
timestamp = 2014-01-30D09:00:00.049089600
before open market
800
timestamp = 2014-01-30D09:00:00.049089600
before open market
801
timestamp = 2014-01-30D09:00:00.049089600
before open market
802
timestamp = 2014-01-30D09:00:00.049089600
before open market
803
timestamp = 2014-01-30D09:00:00.049089600
before open market
804
timestamp = 2014-01-30D09:00:00.049089600
before open market
805
timestamp = 2014-01-30D09:00:00.049089600
before open market
806
timestamp = 2014-01-30D09:00:00.049089600
before open market
807
timestamp = 2014-01-30D09:00:00.049089600
before open market
808
timestamp = 2014-01-30D09:00:00.049089600
before open market
809
timestamp = 2014-01-30D09:00:00.049089600
before open market
810
timestamp = 2014-01-30D09:00:00.095089600
before open market
811
timestamp = 2014-01-30D09:00:00.101089600
before open market
812
119
timestamp = 2014-01-30D09:00:00.101089600
before open market
813
timestamp = 2014-01-3

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:510: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


817
timestamp = 2014-01-30D09:00:00.488076600
before open market
818
timestamp = 2014-01-30D09:00:00.489076600
before open market
819
timestamp = 2014-01-30D09:00:00.530076600
before open market
820
timestamp = 2014-01-30D09:00:00.536076600
before open market
821
timestamp = 2014-01-30D09:00:00.536076600
before open market
822
timestamp = 2014-01-30D09:00:00.677076600
before open market
823
timestamp = 2014-01-30D09:00:00.680076600
before open market
824
timestamp = 2014-01-30D09:00:00.717076600
before open market
825
timestamp = 2014-01-30D09:00:00.717076600
before open market
826
timestamp = 2014-01-30D09:00:00.730076600
before open market
827
121
timestamp = 2014-01-30D09:00:00.744076600
before open market
828
122
timestamp = 2014-01-30D09:00:00.745076600
before open market
829
123
timestamp = 2014-01-30D09:00:00.765076600
before open market
830
timestamp = 2014-01-30D09:00:01.508063600
position and Best Price is True
831
timestamp = 2014-01-30D09:00:01.511063600
position and Best P

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:444: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1008
timestamp = 2014-01-30D09:04:03.012930600
position and Best Price is True
1009
208
timestamp = 2014-01-30D09:04:03.012930600
position and Best Price is True
1010
timestamp = 2014-01-30D09:04:03.012930600
position and Best Price is True
1011
209
timestamp = 2014-01-30D09:04:03.014930600
position and Best Price is True
1012
timestamp = 2014-01-30D09:04:03.014930600
position and Best Price is True
1013
timestamp = 2014-01-30D09:04:03.015930600
position and Best Price is True
1014
timestamp = 2014-01-30D09:04:03.184930600
position and Best Price is True
1015
timestamp = 2014-01-30D09:04:03.184930600
position and Best Price is True
1016
timestamp = 2014-01-30D09:04:03.184930600
position and Best Price is True
1017
timestamp = 2014-01-30D09:04:03.184930600
position and Best Price is True
1018
timestamp = 2014-01-30D09:04:03.184930600
position and Best Price is True
1019
timestamp = 2014-01-30D09:04:03.187930600
position and Best Price is True
1020
timestamp = 2014-01-30D09:04:03.1879306

In [52]:
order_book_ask_sum,order_book_ask_sum[0:1].values.tolist()[0][1],data[j-1:j].BestQuantity.iloc[0]

(     Price  QuantityDifference
 0   671500                  55
 1   672000                 111
 2   672500                  70
 3   673000                   4
 4   673500                   4
 5   674000                 189
 6   674500                  83
 7   675000                 131
 8   675500                  31
 9   676000                  23
 10  676500                  31
 11  677000                  49
 12  677500                   7
 13  678000                 251
 14  678500                   5
 15  679000                  31
 16  679500                   1
 17  680000                  16
 18  680500                   1
 19  681000                   4
 20  681500                 301
 21  682000                   4
 22  682500                   2
 23  683000                   1
 24  683500                   2
 25  684000                   8
 26  684500                   4
 27  685000                   2
 28  685500                   1
 29  686000                 401
 ..     

In [60]:
j

1210

In [61]:
#order_book_ask_sum#[0:1].values.tolist()[0][1]

In [67]:
data[4027:4030]

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity,QuantityDifference_
4027,CNG14,4028,2014-01-30D09:22:16.388743200,F4396A0D50F03410,1,672500,-1,T,B,672000,111,-1
4028,CNG14,4029,2014-01-30D09:22:16.389743200,F4B96A0D5C453F65,1,672500,-1,T,B,672000,111,-1
4029,CNG14,4030,2014-01-30D09:22:16.389743200,0BCE4976EEE7D147,1,672500,-2,T,B,672000,111,-2


In [37]:
order_book_bid_sum[0:1].values.tolist()[0][1] == data[i-1:i].BestQuantity.iloc[0]

False

In [38]:
data[300:301]

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity,QuantityDifference_
300,CNG14,301,2014-01-30D08:54:03.575710600,D4EAE24552613141,8,674000,1,,B,674000,10,1


In [32]:
pd.read_table('order_book.csv',sep=',')#[0:50]

,Bid,Bid_Quantity,Ask,Ask_Quantity
0,TimeStamp,2014-01-30D04:27:08.581903900,NaN,NaN
1,665000.0,150.0,682000.0,1.0
2,663500.0,1.0,683500.0,1.0
3,663000.0,10.0,689000.0,100.0
4,660000.0,100.0,691000.0,10.0
5,NaN,NaN,699000.0,10.0
6,NaN,NaN,700000.0,1.0
7,NaN,NaN,717500.0,10.0
8,TimeStamp,2014-01-30D08:45:00.361550600,NaN,NaN
9,668000.0,1.0,682000.0,1.0


In [45]:
data[100:120]

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity,QuantityDifference_
100,CNG14,101,2014-01-30D08:49:34.924201600,87564D661B3BFD5C,4,675000,2,,A,675000,153,2
101,CNG14,102,2014-01-30D08:49:39.903136600,5821EE451C2AFF4B,41,631500,20,,B,675000,10,20
102,CNG14,103,2014-01-30D08:49:41.189110600,FAEAC645AD358C35,5,675000,6,,A,675000,159,6
103,CNG14,104,2014-01-30D08:49:53.764954600,F6296A45687949B9,2,675000,1,,A,675000,160,1
104,CNG14,105,2014-01-30D08:50:12.905707600,8324E92B630F454F,29,665000,1,,B,675000,10,1
105,CNG14,106,2014-01-30D08:50:20.867603600,50B9624540CD23ED,7,675000,1,,A,675000,161,1
106,CNG14,107,2014-01-30D08:50:29.121495300,7C22E701636347A3,3,675000,70,,A,674500,72,70
107,CNG14,108,2014-01-30D08:50:29.121495300,FA6ACA4D2D540C54,1,675000,-1,,B,674500,10,-1
108,CNG14,109,2014-01-30D08:50:29.121495300,FA6ACA4D2D540C54,1,674500,1,,B,674500,10,1
109,CNG14,110,2014-01-30D08:50:29.121495300,74224A0DC315A215,2,675000,-1,,B,674500,10,-1


In [179]:
temp_ask,temp_bid

(2, 14)

In [180]:
order_book_ask[0],order_book_bid[0]

(    Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  682000  0F50C53613C0F881                   1                    1
 1  683500  D8F8E619640F43AF                   1                    1
 2  689000  C4F76F586C344E54                 100                  100
 3  691000  6EA6E34859783E78                  10                   10
 4  699000  662E6B4847A22CA2                  10                   10
 5  700000  BFC92EA46AE44FA4                   1                    1
 6  717500  2BD5C16638561AB6                  10                   10,
     Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  665000  48714F4C6C254E45                  50                   50
 1  665000  C0F94F4C3F5B217B                  50                   50
 2  665000  4071CF4C6EE55105                  50                   50
 3  663500  FE234A199A1979B9                   1                    1
 4  663000  27586D726EF753F7                  10                   10
 5  660000  6CB1630

In [181]:
order_book_ask[1],order_book_bid[1]

(    Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  682000  0F50C53613C0F881                   1                    1
 1  683500  D8F8E619640F43AF                   1                    1
 2  689000  C4F76F586C344E54                 100                  100
 3  691000  6EA6E34859783E78                  10                   10
 4  699000  662E6B4847A22CA2                  10                   10
 5  700000  BFC92EA46AE44FA4                   1                    1
 6  717500  2BD5C16638561AB6                  10                   10
 7  725000  7A666A457D4F5C2F                   4                    4,
     Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  668000  2DD0417EFB04DFC4                   1                    1
 1  665000  48714F4C6C254E45                  50                   50
 2  665000  C0F94F4C3F5B217B                  50                   50
 3  665000  4071CF4C6EE55105                  50                   50
 4  663500  FE234A1

In [182]:
order_book_ask[2],order_book_bid[2]

(    Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  682000  0F50C53613C0F881                   1                    1
 1  682000  83A8692BEE72D0B2                   1                    1
 2  683500  D8F8E619640F43AF                   1                    1
 3  689000  C4F76F586C344E54                 100                  100
 4  691000  6EA6E34859783E78                  10                   10
 5  699000  662E6B4847A22CA2                  10                   10
 6  700000  BFC92EA46AE44FA4                   1                    1
 7  717500  2BD5C16638561AB6                  10                   10
 8  725000  7A666A457D4F5C2F                   4                    4,
     Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  668000  2DD0417EFB04DFC4                   1                    1
 1  665500  8142C93656273847                   1                    1
 2  665000  48714F4C6C254E45                  50                   50
 3  665000  C0F94F4

In [32]:
order_book_ask[temp_ask],order_book_bid[temp_bid]

(      Price       OrderNumber  QuantityDifference  QuantityDifference_
 0    668000  5872EF19009DE4DE                   5                   10
 1    668000  0B88CD76947A795A                  10                   10
 2    668000  23346D6BF357D597                   1                    1
 3    668000  2B114566DBD1C091                   1                    1
 4    668000  DEA1C3197F185E58                  10                   10
 5    668000  F63A621D1E32FDF3                  78                   78
 6    668000  F43AEA1DEA94C974                  20                   20
 7    668000  52268A8986FD677D                   8                    8
 8    668000  8DF0E563D3DFB61F                   8                    8
 9    668000  5AA60A81376A17EA                   8                    8
 10   668000  D03DE65D415B229B                   2                    2
 11   668000  F4320EC980AB612B                   8                    8
 12   668500  A382C93E4E1B331B                  45              

In [52]:
order_book_ask[0][['Price','QuantityDifference']][1:2].values.tolist()[0]

[683500, 1]

In [25]:
import csv
import sys

In [74]:
import csv
with open('order_book.txt', 'wb') as csvfile:
    f = csv.writer(csvfile)#, delimiter=' ',quotechar='|', quoting=csv.QUOTE_MINIMAL)

In [53]:
with open('eggs.txt','a') as f:
    spamwriter = csv.writer(f,delimiter=',',)
    spamwriter.writerow(order_book_ask[0][['Price','QuantityDifference']][0:1].values.tolist()[0])
    spamwriter.writerow(order_book_ask[0][['Price','QuantityDifference']][1:2].values.tolist()[0])
    spamwriter.writerow(order_book_ask[0][['Price','QuantityDifference']][2:3].values.tolist()[0])
    spamwriter.writerow(order_book_ask[0][['Price','QuantityDifference']][3:4].values.tolist()[0])
    spamwriter.writerow(order_book_ask[0][['Price','QuantityDifference']][4:5].values.tolist()[0])

In [96]:
with open('order_book.txt','a') as f:
    order_book = csv.writer(f)
    order_book.writerow([data.TimeStamp[i]])
    order_book = csv.writer(f,delimiter=',')
    for i in range(1,max(len(order_book_ask[0]),len(order_book_bid[0]))):
        order_book.writerow(a[i:i+1].values.tolist()[0])
        

In [81]:
data.TimeStamp[0]

'2014-01-30D04:27:08.581903900'

In [135]:
#a = pd.concat([order_book_bid[1][['Price','QuantityDifference']],order_book_ask[1][['Price','QuantityDifference']]],axis = 1)

In [134]:
#a

In [133]:
#pd.read_table('eggs.txt')#,sep = ',')

In [19]:
with open('order_book.csv', 'rb') as f:
    reader = csv.reader(f)
    for row in reader:
        print row

In [5]:
import time
start = time.time()
order_book_bid_merge = []
order_book_ask_merge = []
for i in range(0,len(order_book_bid)):
    a = order_book_bid[i][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()#.sort(['Price'],ascending = [False])
    order_book_bid_merge.append(pd.DataFrame(a.values))
for i in range(0,len(order_book_ask)):
    a = order_book_ask[i][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()#.sort(['Price'],ascending = [True])
    order_book_ask_merge.append(pd.DataFrame(a.values))
end = time.time()            
print ("Total time = %f"%(end - start)) 


Total time = 0.432648


In [17]:
order_book_bid_merge[7]

,0,1
0,660000,100
1,663000,10
2,663500,1
3,664000,1
4,665000,151
5,665500,1
6,666000,1
7,668000,1


In [10]:
order_book_ask_merge[0]

,0,1
0,682000,1
1,683500,1
2,689000,100
3,691000,10
4,699000,10
5,700000,1
6,717500,10


In [20]:
import time
start = time.time()
time_ = data.TimeStamp.unique()
time_bid = data[(data.BidOrAsk == 'B')].TimeStamp.unique()[1:]
time_ask = data[(data.BidOrAsk == 'A')].TimeStamp.unique()[1:]
bid_index = 0
ask_index = 0
a = 0
b = 0
first = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
first = pd.DataFrame(first.values)
time = pd.DataFrame([['TimeStamp',time_[0][11:29],'','']])
order_book_concat = time.append(first).reset_index(drop=True)
order_book = order_book_concat
#print order_book
time_series = 1
for time in time_[1:]:
    #print 'time' + time
    #print 'time_series = %d'%(time_series)
    time_to_concat = pd.DataFrame([['TimeStamp',time_[time_series][11:29],'','']])
    #print time_to_concat
    #print time_bid[a]
    #print time_ask[b]
    #print order_book_bid_merge[a]
    #print order_book_ask_merge[b]
    if time == time_bid[a] and time == time_ask[b]:
        #print '1 = ' + time
        bid_index = bid_index + 1
        ask_index = ask_index + 1
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        a = a + 1
        b = b + 1
    elif time == time_bid[a] and time != time_ask[b]: 
        #print '2 = ' +time
        bid_index = bid_index + 1        
        
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        a = a + 1
    elif time != time_bid[a] and time == time_ask[b]: 
        #print '3 = ' +time
        ask_index = ask_index + 1    
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        b = b + 1
    else:
        continue
    order_book = pd.concat([order_book,order_book_concat])    
    time_series = time_series + 1    
    
    #print order_book
end = time.time()            
print "Total time = %f"%(end - start) 


IndexError: list index out of range

In [31]:
order_book = pd.DataFrame(order_book.values)
order_book.to_csv('CNF14_01_03_order_book_final.csv',sep = '\t',encoding = 'utf-8')

In [30]:
import time
start = time.time()


time_ = data.TimeStamp.unique()
time_bid = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
time_ask = data[(data.BidOrAsk == 'A')].TimeStamp.unique()

bid_index = 0
ask_index = 0
a = 0
b = 0
first = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
first = pd.DataFrame(first.values)
time = pd.DataFrame([['TimeStamp',time_[0][11:29],'','']])
order_book_concat = time.append(first).reset_index(drop=True)
order_book = order_book_concat
#print order_book
a = 1
b = 1
time_series = 1
for time in time_[1:]:
    #print 'time' + time
    #print 'time_series = %d'%(time_series)
    time_to_concat = pd.DataFrame([['TimeStamp',time_[time_series][11:29],'','']])
    #print time_to_concat
    #print time_bid[a-1]
    #print time_ask[b-1]    
    #print time_bid[a]
    #print time_ask[b]
    #print order_book_bid_merge[a]
    #print order_book_ask_merge[b]
    if time == time_bid[a] and time == time_ask[b]:
        #print '1 = ' + time
        #a = a + 1
        #b = b + 1
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[a].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[b].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        a = a + 1
        b = b + 1
    elif time == time_bid[a] and time != time_ask[b]: 
        #print '2 = ' +time
        ##a = a + 1
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[a].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[b-1].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        a = a + 1
        
    elif time != time_bid[a] and time == time_ask[b]: 
        #print '3 = ' +time
        ##b = b + 1
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[a-1].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[b].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        b = b + 1
    else:
        continue
    order_book = pd.concat([order_book,order_book_concat])    
    time_series = time_series + 1    
    #print order_book
end = time.time()            
print "Total time = %f"%(end - start) 


IndexError: index 45578 is out of bounds for axis 0 with size 45578

In [7]:
order_book = pd.DataFrame(order_book.values)
order_book.to_csv('CNF14_01_13_order_book_final.csv',sep = '\t',encoding = 'utf-8')